In [9]:
import random
import math
import os

In [3]:
# Referencia: https://github.com/jchen2186/rsa-implementation/blob/master/rsa.py
def _gcd(a, b):
    if (b == 0):
        return a
    else:
        return _gcd(b, a % b)
    
def _ext_euc(a, b):
    """
    Performs the extended Euclidean algorithm
    Returns the gcd, coefficient of a, and coefficient of b
    """
    x, old_x = 0, 1
    y, old_y = 1, 0

    while (b != 0):
        quotient = a // b
        a, b = b, a - quotient * b
        old_x, x = x, old_x - quotient * x
        old_y, y = y, old_y - quotient * y

    return a, old_x, old_y


In [4]:
def _phi(p,q):
    return (p-1)*(q-1)

# Referencia: clases

def _potencia_perfecta(a, b):
    while a % b == 0:
        a = a / b
        if a == 1:
            return True
    return False

import math
def  _es_potencia_(n, i, a , b):
    if a >= b:
        return False
    n_ = ((a+b)/2) ** i
    if b-a <=1:
        if a**i == n or b**i==n:
            return True
        return False
    elif n_ == n:
        return True
    elif n_ < n:
        return _es_potencia_(n,i, a = math.ceil((a+b)/2), b =b )
    elif n_ > n:
        return _es_potencia_(n, i, a = a, b= math.floor((a + b)/2) )
    return False
        
def _es_potencia(n):
    for i in range(2, math.ceil(math.log(n, 2))+1):
        if _es_potencia_(n, i, 0 , u):
            return True
    return False
        

def _test_primalidad(n: int, k: int):
    if n==1:
        return False
    elif n==2:
        return False
    # elif _es_potencia(n):
    #     return False
    else:
        neg = 0
        for i in range(1, k+1):
            a = random.randint(2,n-1)
            if _gcd(a,n) > 1:
                return False
            else:
                b = pow(a, (n-1)//2, n)
                if b == n-1:
                    neg +=1
                elif b != 1:
                    return False
        if neg >0:
            return True
        return False

In [17]:
def _random_with_N_digits(n):
    while True:
        b = bytearray(os.urandom(math.ceil(n/8) + 3))
        nb = int.from_bytes(b, 'big')
        if  nb.bit_length() > n:
            return nb

def _generate_random_prime(N, n):
    primes = list()
    for i in range(n):
        while True:
            r = _random_with_N_digits(N)
            if  _test_primalidad(r, 50):
                primes.append(r)
                break
    return primes

def _get_d(phi):
    while True:
        r = random.randint(2, phi)
        if _gcd(r, phi) == 1:
            return r
        
        
def _generate_keys(N):
    P,Q = _generate_random_prime(int(N/2), 2)
    N = P*Q
    phi = (P-1)*(Q-1)
    d = _get_d(phi)
    gcd, x, y = _ext_euc(d, phi)
    e = x + phi if x<0 else x
    return e, d, P*Q

In [18]:
class RSAReceiver:
    def __init__(self, bit_len): 
        """
        Arguments:
        bit_len: A lower bound for the number of bits of N,
        the second argument of the public and secret key.
        """
        self.keys = _generate_keys(bit_len)
        self.e = self.keys[0]
        self.d = self.keys[1]
        self.N = self.keys[2]
        
    def get_public_key(self):
        """
        Returns:
        public_key:
            Public key expressed as a Python ’bytearray’ using the PEM format. This means the public key is divided in:
            (1) The number of bytes of e (4 bytes)
            (2) the number e (as many bytes as indicated in (1))
            (3) The number of bytes of N (4 bytes)
            (4) the number N (as many bytes as indicated in (3))
        """
        n_e = math.ceil(self.e.bit_length() / 8)
        n_e_bytes = n_e.to_bytes(4, byteorder = 'big')
        e_bytes = self.e.to_bytes(n_e, byteorder = 'big')
        
        n_N = math.ceil(self.N.bit_length() / 8)
        n_N_bytes = n_N.to_bytes(4, byteorder = 'big')
        N_bytes = self.N.to_bytes(n_N, byteorder = 'big')
        return n_e_bytes + e_bytes + n_N_bytes + N_bytes

    def decrypt(self, ciphertext):
        """
            Arguments:
            ciphertext: The ciphertext to decrypt Returns:
            message: The original message """
        decrypted = bytearray()
        n = self._get_n(self.N)
        for block in self._chunker(ciphertext, n + 1):
            decrypted += self._decrypt_block(block,n)
        return decrypted.decode('utf-8').rstrip('\x00')
    
    def _decrypt_block(self, block, n):
        int_block = int.from_bytes(block, 'big')
        return pow(int_block, self.d, self.N).to_bytes(n, byteorder = 'big')
        
    def _get_n(self, N):
        bites_N = N.bit_length()
        return math.floor((bites_N-1)/8)
        
    def _chunker(self, seq, size):
        # https://stackoverflow.com/questions/434287/what-is-the-most-pythonic-way-to-iterate-over-a-list-in-chunks
        return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [19]:
class RSASender:
    def __init__(self, public_key):
        """
        Arguments:
        public_key: The public key that will be used to encrypt messages """
        self.key = self._read_key(public_key)
        self.e = self.key[0]
        self.N = self.key[1]
        
    def encrypt(self, message: str):
        """
        Arguments:
        message: The plaintext message to encrypt Returns:
        ciphertext: The encrypted message """
        byte_message = bytearray(message, 'utf-8')
        encrypted = bytearray()
        n = self._get_n(self.N)
        for block in self._chunker(byte_message, n):
            encrypted += self._encrypt_block(block,n)
        return encrypted
        
    def _get_n(self, N):
        # bits_N = len(bytes_N) * 8
        bites_N = N.bit_length()
        return math.floor((bites_N-1)/8)
    
    def _chunker(self, seq, size):
        # https://stackoverflow.com/questions/434287/what-is-the-most-pythonic-way-to-iterate-over-a-list-in-chunks
        return (seq[pos:pos + size] for pos in range(0, len(seq), size))
    
    def _encrypt_block(self, block, n):
        int_block = int.from_bytes(block, 'big')
        me = pow(int_block, self.e, self.N)
        if len(block) == n:
            return bytearray(me.to_bytes(n + 1, byteorder = 'big'))
        else:
            return bytearray(me.to_bytes(n+1, byteorder = 'big'))
        
    def _read_key(self, public_key):
        n_e = int.from_bytes(public_key[:4], 'big')
        e = int.from_bytes(public_key[4: n_e+4], 'big')
        N_n = int.from_bytes(public_key[n_e+4: (n_e+4)+4], 'big')
        N = int.from_bytes(public_key[-N_n:], 'big') 
        return (e, N)
        
        
        

In [25]:
# rec = RSAReceiver(2048)
# public_key = rec.get_public_key()
# sender = RSASender(public_key)
# message = 'Este mensaje es bien secreto y esperamos que nadie logre encontrarlo'
# text = (
#     'Being open source means anyone can independently review the code. If it was closed source, nobody could verify the security. I think it’s essential for a program of this nature to be open source.'
# )
# encrypted_m = sender.encrypt(text)
# decrypted = rec.decrypt(encrypted_m)
# print(decrypted)

In [26]:
# public_key = b64decode('AAAAQQGHaihgiufnjzyLXufDjUCGuaHrsUL+hCF/pMFHPoh+ZVi/2bMFh6oelzElVklsJ9mglyQjJIKAb1JB9mvtaEkLAAAAQQHIuF+wIJw6uzq8uXpW/QmsNjtBJ8HCJJcu2h7sDX18nc2qWYDWTfMiXPmPRvhkkz4A0oXTAMDP9xsxUIjYQNsx')
# text = (
#     'Being open source means anyone can independently review the code. If it was closed source, nobody could verify the security. I think it’s essential for a program of this nature to be open source.'
# )
# sender = RSASender(public_key)
# cipher = sender.encrypt(text)
# print(b64encode(cipher))
# b'ALwPm7JXWbqGeIflV8PYgprs6mSgCH2Ydy0rgvFolzY0mczKItlPSHueL54uvDJXIz9pXoHZGAOPWVYYbcwRh3EBl8pi3MraUC2BBFUviMPFwNMwza/QMd5DNG9tH8doHlLRRt+15wLrsIE+m5T8fuM4HHixSNcEoOdN8T++q0PkzQDXL+UgbusiD3J+QPO59aqAB5HFcZ7P5U3fhFS8Qm1vLG8vlIulCby0jGLgjTtLUhFD/QhAof0y4F20gxedQDHwAOIrz6PEoBWnHmwLU0QNN0Rs542RvJ8BeEGhBDS5ZvD0/0Ix3ZqKT6HtP4ugfPD75/5LYGioJBwrg2DXbQucFj8='